## Import Packages 

In [2]:
import cv2
import time
import numpy as np

## Creating Output File and Importing Static Image

In [3]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('Akatsuki.avi',fourcc,20.0,(640,480))
static_img = cv2.imread('static.jpg')

## Replacing the Dynamic Object's Area with Blended Image

In [5]:
#Create a background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()
cap=cv2.VideoCapture(0)
time.sleep(5)
count=0
background=0

#Recording the background image for the purpose of blending it with static image
for i in range(60):
    ret,background=cap.read()
background=np.flip(background, axis=1) 

#Resize the static image to blend with background recorded
static_img = cv2.resize(static_img, (640,480), interpolation = cv2.INTER_AREA) 
blend = cv2.addWeighted(background,0.8,static_img,0.8,0)                     #Blended image of background and static for the purpose of replacing it in masked area

while (cap.isOpened()):
    ret,img=cap.read()
    if not ret:                                                              #Case when False is returned by cap.read()
        break
    count+=1
    img=np.flip(img,axis=1)
    
    mask1 = backSub.apply(img)                                               #Applying background subtraction method to get a dynamic object in the frame
    
    #Handle noise in the image using morphologyEx
    mask1=cv2.morphologyEx(mask1, cv2.MORPH_OPEN,np.ones((3,3),np.uint8))    #Erosion followed by Dilation
    mask1=cv2.morphologyEx(mask1, cv2.MORPH_DILATE,np.ones((3,3),np.uint8))  #Dilation
    mask1=cv2.morphologyEx(mask1, cv2.MORPH_DILATE,np.ones((3,3),np.uint8))  #Dilation
    mask2=cv2.bitwise_not(mask1)                                             #Obtain Inverted mask of mask1
    res1=cv2.bitwise_and(img,img,mask=mask2)                                 #Applying the total frame i.e.img over the masked region by bitwise_and so that unmasked area does not contain anything from the current frame.
    res2=cv2.bitwise_and(blend,blend,mask=mask1)                             #Similar to previous step but in this case we pass the blended image in the masked area and unmasked area remains unaffected.
    
    finalOutput=cv2.addWeighted(res1,1,res2,1,0)                             #It blends both Image together, and since both res1 and res2 didn't have anything in unmasked region they add up together without affecting each other.
    out.write(finalOutput)                                                   #Writing final Output produced in the same directory
    cv2.imshow("magic",finalOutput)                                          #Wait for 1ms for user response and if he enters "q" in his/her keyboard, the loop is terminated and webcam window closes.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()                                                      #Destroying all the windows and releasing the cv2 objects created.